In [3]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf 
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [4]:
train_dir = "train"
validation_dir = "test"
BATCH_SIZE = 32
IMG_SIZE = (70, 70)

train_dataset = image_dataset_from_directory(
    train_dir, label_mode = 'categorical', shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE
)

validation_dataset = image_dataset_from_directory(
    validation_dir,label_mode = 'categorical', shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE
)

class_names = train_dataset.class_names


Found 6000 files belonging to 5 classes.
Found 2000 files belonging to 5 classes.


In [5]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

data_augmentation = tf.keras.Sequential(
    [
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
        tf.keras.layers.experimental.preprocessing.RandomContrast(0.5)
    ]
)


In [6]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1.0 / 127.5, offset=-1)

IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch) 

base_model.trainable = False


In [7]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)

base_model.trainable = False

In [8]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)

prediction_layer = tf.keras.layers.Dense(len(class_names))
prediction_batch = prediction_layer(feature_batch_average)

inputs = tf.keras.Input(shape=(70, 70, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

base_learning_rate = 0.0001
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)
model.summary()
len(model.trainable_variables)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 70, 70, 3)]       0         
_________________________________________________________________
sequential (Sequential)      (None, 70, 70, 3)         0         
_________________________________________________________________
tf.math.truediv (TFOpLambda) (None, 70, 70, 3)         0         
_________________________________________________________________
tf.math.subtract (TFOpLambda (None, 70, 70, 3)         0         
_________________________________________________________________
mobilenetv2_1.00_224 (Functi (None, 3, 3, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0     

C:\Users\joel\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


2

In [ ]:
import os 
class_names = os.listdir('train')
from tensorflow.keras.models import load_model
class model:
    def __init__(self):
        self.model = load_model('picture')
    def predict(self, img):
        pred_vec = self.model.predict(img)
        return class_names[np.argmax(pred_vec)]

In [ ]:
pred_vec = mdl.model.predict(image_batch.numpy()[[0]])

In [11]:
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf 
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [17]:
def train_model(image_directory = 'images', training_epochs = 1, fine_tune_epochs = 1 ):
    train_dir = "train"
    validation_dir = "test"
    BATCH_SIZE = 32
    IMG_SIZE = (70, 70)
    total_epochs = training_epochs + fine_tune_epochs

    train_dataset = image_dataset_from_directory(
        train_dir, label_mode = 'categorical', shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)

    validation_dataset = image_dataset_from_directory(
        validation_dir, label_mode = 'categorical', shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)

    class_names = train_dataset.class_names
   
    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],)
    
    mdl = model.fit(train_dataset, epochs= total_epochs, validation_data=validation_dataset)
    
    base_model.trainable = True
    fine_tune_at = 100
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False
        
    model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate / 10),
        metrics=["accuracy"],
    )
    mdl_fine = model.fit( train_dataset, epochs=total_epochs, initial_epoch= mdl.epoch[-1], validation_data=validation_dataset
    )
    
    model.save('trained_model')

In [18]:
train_model()

Found 6000 files belonging to 5 classes.
Found 2000 files belonging to 5 classes.
Epoch 1/2
188/188 [==============================] - 112s 537ms/step - loss: 0.7136 - accuracy: 0.7457 - val_loss: 0.4273 - val_accuracy: 0.8485
Epoch 2/2
188/188 [==============================] - 90s 475ms/step - loss: 0.5267 - accuracy: 0.8072 - val_loss: 0.3438 - val_accuracy: 0.8845
Epoch 2/2
188/188 [==============================] - 70s 346ms/step - loss: 0.4092 - accuracy: 0.8518 - val_loss: 0.2856 - val_accuracy: 0.9070


C:\Users\joel\anaconda3\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: trained_model\assets


In [22]:
from Train import train_model

train_model()

Found 6000 files belonging to 5 classes.
Found 2000 files belonging to 5 classes.
Found 6000 files belonging to 5 classes.
Found 2000 files belonging to 5 classes.


C:\Users\joel\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/2
188/188 [==============================] - 76s 383ms/step - loss: 2.1715 - accuracy: 0.3347 - val_loss: 1.2274 - val_accuracy: 0.5550
Epoch 2/2
188/188 [==============================] - 46s 247ms/step - loss: 1.3961 - accuracy: 0.5165 - val_loss: 0.8738 - val_accuracy: 0.6820
Epoch 2/2
188/188 [==============================] - 66s 332ms/step - loss: 0.8431 - accuracy: 0.6923 - val_loss: 0.4328 - val_accuracy: 0.8555


C:\Users\joel\anaconda3\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: trained_model\assets
